In [1]:
!pip install duckdb --no-index --find-links=/kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg
!pip install python-gdcm
!pip install pylibjpeg
!pip install pylibjpeg-libjpeg==2.2.0
!pip install pylibjpeg-openjpeg==2.3.0
!pip install matplotlib==3.10.3
!pip install scikit-learn==1.7.0
!pip install polars --no-index --find-links=/kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/polars_pkg

Looking in links: /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg
Processing /kaggle/input/polars-and-duckdb/kaggle/working/mysitepackages/duck_pkg/duckdb-0.8.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 26.4 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 20.2 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 3.1 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 

In [2]:
!pip install pydicom

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 26.1 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
import polars as pl
import duckdb as dd
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
from pydicom import dcmread
import pickle
import gc
import ctypes
from pathlib import Path
import logging
import json
import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import log_loss
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow import keras

import warnings
warnings.filterwarnings('ignore')

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

print("Number of accelerators: ", tpu_strategy.num_replicas_in_sync)

In [4]:
pl.Config(fmt_str_lengths=1000)
pl.Config.set_tbl_rows(1000)

polars.config.Config

# Load the metadata of the training images
## Also separate out the localizer coordinates into individual columns

In [8]:
train_meta_data = pl.read_csv('/kaggle/input/rsna-intracranial-aneurysm-detection/train.csv'\
                              , low_memory=True)

train_locale_meta_data = pl.read_csv('/kaggle/input/rsna-intracranial-aneurysm-detection/train_localizers.csv'\
                              , low_memory=True)

def parse_coordinates(coord_str):
    if coord_str is None:
        return None, None
    try:
        coord_dict = json.loads(coord_str.replace("'", '"'))
        return float(coord_dict.get('x', 0.0)), float(coord_dict.get('y', 0.0)), int(coord_dict.get('f', 0.0))
    except (json.JSONDecodeError, KeyError, ValueError, AttributeError):
        return None, None

train_locale_meta_data = train_locale_meta_data.with_columns([
    pl.col("coordinates")
    .map_elements(lambda x: parse_coordinates(x)[0], return_dtype=pl.Float64)
    .cast(pl.Float64)
    .alias("coordinates_x"),
    
    pl.col("coordinates")
    .map_elements(lambda x: parse_coordinates(x)[1], return_dtype=pl.Float64)
    .cast(pl.Float64)
    .alias("coordinates_y"),
    
    pl.col("coordinates")
    .map_elements(lambda x: parse_coordinates(x)[2], return_dtype=pl.Int32)
    .cast(pl.Int32)
    .alias("coordinates_f")
])

print("Train CSV shape : ", train_meta_data.shape)
print("Train Localizers CSV shape : ", train_locale_meta_data.shape)
# Show the first few rows
print(train_locale_meta_data.filter(pl.col('coordinates_f') != 0.0)\
      .select(["coordinates", "coordinates_x", "coordinates_y", "coordinates_f"]).head(5))

Train CSV shape :  (4348, 18)
Train Localizers CSV shape :  (2251, 7)
shape: (5, 4)
┌────────────────────────────────┬───────────────┬───────────────┬───────────────┐
│ coordinates                    ┆ coordinates_x ┆ coordinates_y ┆ coordinates_f │
│ ---                            ┆ ---           ┆ ---           ┆ ---           │
│ str                            ┆ f64           ┆ f64           ┆ i32           │
╞════════════════════════════════╪═══════════════╪═══════════════╪═══════════════╡
│ {'x': 272.41014419367286, 'y': ┆ 272.410144    ┆ 194.117971    ┆ 58            │
│ 194.11797116126544, 'f': 58}   ┆               ┆               ┆               │
│ {'x': 262.09232271634613, 'y': ┆ 262.092323    ┆ 208.984615    ┆ 8             │
│ 208.98461538461538, 'f': 8}    ┆               ┆               ┆               │
│ {'x': 287.35893155258765, 'y': ┆ 287.358932    ┆ 205.382304    ┆ 74            │
│ 205.38230383973288, 'f': 74}   ┆               ┆               ┆               │
│ {

In [ ]:
train_meta_data.head(10)

In [9]:
train_locale_meta_data.head(10)

SeriesInstanceUID,SOPInstanceUID,coordinates,location,coordinates_x,coordinates_y,coordinates_f
str,str,str,str,f64,f64,i32
"""1.2.826.0.1.3680043.8.498.10005158603912009425635473100344077317""","""1.2.826.0.1.3680043.8.498.10775329348174902199350466348663848346""","""{'x': 258.3621186176837, 'y': 261.359900373599}""","""Other Posterior Circulation""",258.362119,261.3599,0
"""1.2.826.0.1.3680043.8.498.10022796280698534221758473208024838831""","""1.2.826.0.1.3680043.8.498.53868409774237283281776807176852774246""","""{'x': 194.87253141831238, 'y': 178.32675044883302}""","""Right Middle Cerebral Artery""",194.872531,178.32675,0
"""1.2.826.0.1.3680043.8.498.10023411164590664678534044036963716636""","""1.2.826.0.1.3680043.8.498.24186535344744886473554579401056227253""","""{'x': 189.23979878597123, 'y': 209.19184886465828}""","""Right Middle Cerebral Artery""",189.239799,209.191849,0
"""1.2.826.0.1.3680043.8.498.10030095840917973694487307992374923817""","""1.2.826.0.1.3680043.8.498.75217084841854214544099244823406151875""","""{'x': 208.2805049088359, 'y': 229.78962131837307}""","""Right Infraclinoid Internal Carotid Artery""",208.280505,229.789621,0
"""1.2.826.0.1.3680043.8.498.10034081836061566510187499603024895557""","""1.2.826.0.1.3680043.8.498.71237104731452368587327801789352569583""","""{'x': 249.86745590416498, 'y': 220.623044646393}""","""Anterior Communicating Artery""",249.867456,220.623045,0
"""1.2.826.0.1.3680043.8.498.10035643165968342618460849823699311381""","""1.2.826.0.1.3680043.8.498.30083322134992576720636296296008654053""","""{'x': 223.66020358711827, 'y': 225.3447011195274}""","""Right Anterior Cerebral Artery""",223.660204,225.344701,0
"""1.2.826.0.1.3680043.8.498.10035643165968342618460849823699311381""","""1.2.826.0.1.3680043.8.498.46752468449107005352755579756056225731""","""{'x': 289.2376764288231, 'y': 211.78100912436958}""","""Left Middle Cerebral Artery""",289.237676,211.781009,0
"""1.2.826.0.1.3680043.8.498.10035643165968342618460849823699311381""","""1.2.826.0.1.3680043.8.498.14504961303532677815336995545201912867""","""{'x': 232.88987039309842, 'y': 226.4305401310092}""","""Right Supraclinoid Internal Carotid Artery""",232.88987,226.43054,0
"""1.2.826.0.1.3680043.8.498.10042423585566957032411171949972906248""","""1.2.826.0.1.3680043.8.498.63062558671948377310688873437936911318""","""{'x': 140.1086940862857, 'y': 177.63838968827815}""","""Right Middle Cerebral Artery""",140.108694,177.63839,0


In [10]:
# Get summary statistics of the new columns
print(train_locale_meta_data.select(["coordinates_x", "coordinates_y", "coordinates_f"]).describe())

shape: (9, 4)
┌────────────┬───────────────┬───────────────┬───────────────┐
│ statistic  ┆ coordinates_x ┆ coordinates_y ┆ coordinates_f │
│ ---        ┆ ---           ┆ ---           ┆ ---           │
│ str        ┆ f64           ┆ f64           ┆ f64           │
╞════════════╪═══════════════╪═══════════════╪═══════════════╡
│ count      ┆ 2251.0        ┆ 2251.0        ┆ 2251.0        │
│ null_count ┆ 0.0           ┆ 0.0           ┆ 0.0           │
│ mean       ┆ 251.193332    ┆ 215.855472    ┆ 2.673478      │
│ std        ┆ 68.775391     ┆ 54.057286     ┆ 13.121975     │
│ min        ┆ 0.339623      ┆ 5.0           ┆ 0.0           │
│ 25%        ┆ 219.547377    ┆ 190.351987    ┆ 0.0           │
│ 50%        ┆ 253.501157    ┆ 210.702417    ┆ 0.0           │
│ 75%        ┆ 280.620123    ┆ 232.727273    ┆ 0.0           │
│ max        ┆ 714.748844    ┆ 583.68        ┆ 115.0         │
└────────────┴───────────────┴───────────────┴───────────────┘


# Get the image metadata from each training series and create a dataframe out of them

In [19]:
allowed_tags = ['BitsAllocated', 'BitsStored', 'Rows', 'Columns', 'FrameOfReferenceUID', 'HighBit', 'ImageOrientationPatient'
                , 'ImagePositionPatient', 'InstanceNumber', 'Modality', 'PhotometricInterpretation'
                , 'PixelRepresentation', 'PixelSpacing', 'PlanarConfiguration', 'RescaleIntercept', 'RescaleSlope'
                , 'RescaleType', 'SamplesPerPixel', 'SliceThickness', 'SpacingBetweenSlices']

## Functions to collect metadata without the image arrays
### We use python slots to reduce the memory foorprint
### Also use multithreading to speed up processing

In [11]:
class DicomRecord:
    """
    Memory-efficient class for storing DICOM metadata using __slots__
    """
    __slots__ = ['folder_name', 'file_name', 'file_path', 'image_shape'] + [
        'BitsAllocated', 'BitsStored', 'Rows', 'Columns', 'FrameOfReferenceUID',
        'HighBit', 'ImageOrientationPatient', 'ImagePositionPatient', 'InstanceNumber',
        'Modality', 'PhotometricInterpretation', 'PixelRepresentation', 'PixelSpacing',
        'PlanarConfiguration', 'RescaleIntercept', 'RescaleSlope', 'RescaleType',
        'SamplesPerPixel', 'SliceThickness', 'SpacingBetweenSlices'
    ]
    
    def __init__(self, folder_name, file_name, file_path, image_shape):
        self.folder_name = folder_name
        self.file_name = file_name
        self.file_path = file_path
        self.image_shape = image_shape
        for tag in self.__slots__[4:]:  
            setattr(self, tag, None)
    
    def to_dict(self):
        return {slot: getattr(self, slot) for slot in self.__slots__}

In [14]:
def process_single_folder(folder_path, allowed_tags):
    """
    Process a single folder of DICOM files and save image arrays
    """
    try:
        data = []
        dcm_files = list(Path(folder_path).glob("*.dcm"))
        folder_name = Path(folder_path).name
        
        for dcm_file in dcm_files:
            try:
                # Read DICOM file
                ds = dcmread(str(dcm_file))
                original_shape = str(ds.pixel_array.shape)
                
                # Create record
                record = DicomRecord(folder_name, dcm_file.name, dcm_file, original_shape)
                
                # Fill in tags
                for tag in allowed_tags:
                    try:
                        value = getattr(ds, tag)
                        if hasattr(value, '__iter__') and not isinstance(value, str):
                            value = str(list(map(str, value)))
                        else:
                            value = str(value)
                        setattr(record, tag, value)
                    except (AttributeError, TypeError):
                        continue
                
                data.append(record.to_dict())
                
            except Exception as e:
                print(f"Error processing file {dcm_file}: {e}")
                continue
                
        return data
        
    except Exception as e:
        print(f"Error processing folder {folder_path}: {e}")
        return []

In [15]:
def create_dicom_dataset(root_folder, allowed_tags, num_processes=None, chunk_size=100):
    """
    Create dataset with metadata DataFrame and memory-mapped image arrays
    """
    root_path = Path(root_folder)
    folders = [f for f in root_path.iterdir() if f.is_dir()]
    
    if not num_processes:
        num_processes = mp.cpu_count()
    
    # Create directories for temporary and array storage
    temp_dir = Path("temp_chunks")
    temp_dir.mkdir(exist_ok=True)
    
    # Create schema
    schema = {
        'folder_name': pl.Utf8,
        'file_name': pl.Utf8,
        'compressed_image_array': pl.Binary,
        'original_shape': pl.Utf8
    }
    schema.update({tag: pl.Utf8 for tag in allowed_tags})
    
    # Process folders in parallel
    with ProcessPoolExecutor(max_workers=num_processes) as executor:
        for i in range(0, len(folders), chunk_size):
            chunk_folders = folders[i:i+chunk_size]
            chunk_data = []
            
            futures = [
                executor.submit(
                    process_single_folder, 
                    str(folder), 
                    allowed_tags
                    #arrays_dir
                )
                for folder in chunk_folders
            ]
            
            for future in tqdm(futures, 
                             desc=f"Processing chunk {i//chunk_size + 1}/{(len(folders)-1)//chunk_size + 1}"):
                chunk_data.extend(future.result())
            
            if chunk_data:
                chunk_df = pl.DataFrame(
                    chunk_data,
                    schema=schema,
                    infer_schema_length=None
                )
                
                chunk_df.write_parquet(
                    temp_dir / f"dicom_metadata_chunk_{i//chunk_size}.parquet",
                    compression="snappy"
                )
                
                del chunk_data
                del chunk_df
    
    # Combine chunks
    print("\nCombining chunks...")
    chunk_files = list(temp_dir.glob("dicom_metadata_chunk_*.parquet"))
    final_df = pl.concat([
        pl.scan_parquet(str(chunk_file))
        for chunk_file in chunk_files
    ]).collect()
    
    # Clean up temporary files
    for f in chunk_files:
        f.unlink()
    temp_dir.rmdir()
    
    return final_df

In [21]:
with tpu_strategy.scope():
    root_folder = "/kaggle/input/rsna-intracranial-aneurysm-detection/series"
    
    try:
        metadata_df = create_dicom_dataset(
            root_folder, 
            allowed_tags, 
            num_processes=mp.cpu_count(),
            chunk_size=192
        )
    except Exception as e:
        print(f"Error: {e}")

Processing chunk 23/23: 100%|██████████| 124/124 [00:25<00:00,  4.80it/s]



Combining chunks...
Error: too many values to unpack (expected 2)


In [24]:
gc.collect()

88088

In [23]:
metadata_df.columns

NameError: name 'metadata_df' is not defined

# Create the full training data
* Standardize the column names
* Bring in the localizer coordinates for the series where aneurysm is present
* Create another column to signify whether aneurysm is shown in a specific image within a series
* There can be cases where some images of a series cannot catch aneurysm presence
* Bringing all the rows at the image file granularity, i.e. if a file has coordinates then it has aneurysm else not

In [ ]:
new_columns = [col.lower().replace(" ", "_") for col in train_meta_data.columns]
train_meta_data.columns = new_columns
print(train_meta_data.columns)

In [ ]:
df_all_coordinates = dd.sql( \
    "select t2.coordinates_x, t2.coordinates_y, t2.coordinates_f, t1.* \
    from df t1 \
    left join train_locale_meta_data t2 \
    on t1.folder_name = t2.SeriesInstanceUID \
    and replace(t1.file_name, '.dcm','') = t2.SOPInstanceUID "\
).pl()

print(df_all_coordinates.shape)
print(df_all_coordinates.columns)

In [ ]:
df_all_data = dd.sql( \
    "select t2.file_name, t2.coordinates_x, t2.coordinates_y \
    , t1.aneurysm_present as aneurysm_present_in_series \
    , case when t2.coordinates_x is not null then 1 else 0 end as aneurysm_present_in_image \
    , t1.seriesinstanceuid, t1.patientage, t1.patientsex, t1.modality \
    , case when t2.coordinates_x is not null then t1.left_infraclinoid_internal_carotid_artery \
    else 0 end as left_infraclinoid_internal_carotid_artery \
    , case when t2.coordinates_x is not null then t1.right_infraclinoid_internal_carotid_artery \
    else 0 end as right_infraclinoid_internal_carotid_artery \
    , case when t2.coordinates_x is not null then t1.left_supraclinoid_internal_carotid_artery \
    else 0 end as left_supraclinoid_internal_carotid_artery \
    , case when t2.coordinates_x is not null then t1.right_supraclinoid_internal_carotid_artery \
    else 0 end as right_supraclinoid_internal_carotid_artery \
    , case when t2.coordinates_x is not null then t1.left_middle_cerebral_artery \
    else 0 end as left_middle_cerebral_artery \
    , case when t2.coordinates_x is not null then t1.right_middle_cerebral_artery \
    else 0 end as right_middle_cerebral_artery \
    , case when t2.coordinates_x is not null then t1.anterior_communicating_artery \
    else 0 end as anterior_communicating_artery \
    , case when t2.coordinates_x is not null then t1.left_anterior_cerebral_artery \
    else 0 end as left_anterior_cerebral_artery \
    , case when t2.coordinates_x is not null then t1.right_anterior_cerebral_artery \
    else 0 end as right_anterior_cerebral_artery \
    , case when t2.coordinates_x is not null then t1.left_posterior_communicating_artery \
    else 0 end as left_posterior_communicating_artery \
    , case when t2.coordinates_x is not null then t1.right_posterior_communicating_artery \
    else 0 end as right_posterior_communicating_artery \
    , case when t2.coordinates_x is not null then t1.basilar_tip \
    else 0 end as basilar_tip \
    , case when t2.coordinates_x is not null then t1.other_posterior_circulation \
    else 0 end as other_posterior_circulation \
    from train_meta_data t1 \
    join df_all_coordinates t2 \
    on t1.SeriesInstanceUID = t2.folder_name" \
).pl()

print("Full training data: ", df_all_data.shape)
print("Full training data columns: ", df_all_data.columns)
print("Aneurysm not present in {0} series".format(df_all_data.filter(pl.col("coordinates_x").is_null()).shape[0]))

print("Aneurysm present in {0} series".format(df_all_data.filter(pl.col("coordinates_x").is_not_null()).shape[0]))

print("Aneurysm not shown in {0} images".format(df_all_data.filter(pl.col("aneurysm_present_in_image")==0).shape[0]))

print("Aneurysm shown in {0} images".format(df_all_data.filter(pl.col("aneurysm_present_in_image")==1).shape[0]))

print(df_all_data.select(["coordinates_x", "coordinates_y"]).describe())

In [ ]:
df_all_data.write_parquet('full_training_data.parquet')

# Looking at a specific CTA image that shows aneurysm

In [ ]:
df_all_data.filter(
    (pl.col("seriesinstanceuid") == '1.2.826.0.1.3680043.8.498.10005158603912009425635473100344077317')
    &
    (pl.col('coordinates_x').is_null())
)

In [ ]:
# Version with zoom functionality
def load_and_view_single_slice_with_zoom(dcm_path, x_coord, y_coord, zoom_size=100):
    """
    Load and display a single DICOM slice with crosshair and zoomed inset
    
    Args:
        dcm_path: Path to the DICOM file
        x_coord: x coordinate for the crosshair
        y_coord: y coordinate for the crosshair
        zoom_size: Size of the zoom window in pixels
    """
    # Read DICOM file
    ds = dcmread(dcm_path)
    img = ds.pixel_array
    
    # Create figure and axes
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))
    
    # Main image with crosshair
    ax1.imshow(img, cmap='gray')
    ax1.axvline(x=x_coord, color='red', alpha=0.5)
    ax1.axhline(y=y_coord, color='red', alpha=0.5)
    ax1.plot(x_coord, y_coord, 'r+', markersize=10, markeredgewidth=2)
    
    # Zoomed region
    x_start = int(max(0, x_coord - zoom_size/2))
    x_end = int(min(img.shape[1], x_coord + zoom_size/2))
    y_start = int(max(0, y_coord - zoom_size/2))
    y_end = int(min(img.shape[0], y_coord + zoom_size/2))
    
    zoomed = img[y_start:y_end, x_start:x_end]
    ax2.imshow(zoomed, cmap='gray')
    
    # Add crosshair to zoomed region
    center_x = x_coord - x_start
    center_y = y_coord - y_start
    ax2.axvline(x=center_x, color='red', alpha=0.5)
    ax2.axhline(y=center_y, color='red', alpha=0.5)
    ax2.plot(center_x, center_y, 'r+', markersize=10, markeredgewidth=2)
    
    ax1.axis('off')
    ax2.axis('off')
    ax1.set_title('Full Image')
    ax2.set_title('Zoomed Region')
    
    plt.tight_layout()
    plt.show()

In [ ]:
dcm_path = '/kaggle/input/rsna-intracranial-aneurysm-detection/series/1.2.826.0.1.3680043.8.498.10005158603912009425635473100344077317/1.2.826.0.1.3680043.8.498.10775329348174902199350466348663848346.dcm'
x_coord = 258.362119
y_coord = 261.3599
load_and_view_single_slice_with_zoom(dcm_path, x_coord, y_coord)

In [ ]:
dicom_ds = dcmread(dcm_path)
print(dicom_ds.pixel_array.shape)

In [ ]:
allowed_tags = ['BitsAllocated', 'BitsStored', 'Rows', 'Columns', 'FrameOfReferenceUID', 'HighBit', 'ImageOrientationPatient'
                , 'ImagePositionPatient', 'InstanceNumber', 'Modality', 'PhotometricInterpretation'
                , 'PixelRepresentation', 'PixelSpacing', 'PlanarConfiguration', 'RescaleIntercept', 'RescaleSlope'
                , 'RescaleType', 'SamplesPerPixel', 'SliceThickness', 'SpacingBetweenSlices']

dicom_ds = dcmread(dcm_path)
for attr in dir(dicom_ds):
    ##if not attr.startswith('_') and attr != 'PixelData':  # Skip private attributes and pixel data
    if attr in allowed_tags:
        try:
            value = getattr(dicom_ds, attr)
            #if not callable(value) and attr != 'pixel_array':
            print(f"{attr}: {value}")
        except AttributeError:
            continue

## Checking the cardinality of the different type of image modalities

In [ ]:
dd.sql("select modality, count(1) as image_count from df_all_data group by modality").pl()

# Checking for different Image array shapes

In [ ]:
root_file_path = '/kaggle/input/rsna-intracranial-aneurysm-detection/series/'

def get_image_array_shape(seriesinstanceuid, dcm_file_name):
    image_path = root_file_path + seriesinstanceuid + '/' + dcm_file_name
    dicom_ds = dcmread(image_path)
    return str(dicom_ds.pixel_array.shape)

vectorized_process = np.vectorize(get_image_array_shape, otypes=[object])

df_all_data = df_all_data.with_columns(
    pl.Series(
        name="img_array_shape_tuple",
        values=vectorized_process(
            df_all_data["seriesinstanceuid"].to_numpy(),
            df_all_data["dcm_file_name"].to_numpy()
        )
    )
)

In [ ]:
temp_df = dd.sql("select modality, img_array_shape_tuple, count(distinct(seriesinstanceuid)) as series_count \
, count(distinct(dcm_file_name)) as image_count \
from df_all_data group by modality, img_array_shape_tuple order by 4 desc").pl()

In [ ]:
grouped_df = temp_df.group_by(['Modality', 'img_array_shape_tuple']).agg([
        pl.col('series_count').sum().alias('total_series'),
        pl.col('image_count').sum().alias('total_images')
    ]).filter(
        (pl.col('total_series') > 20) | (pl.col('total_images') > 20)
    ).sort('total_images', descending=True)

In [ ]:
# Convert to pandas for easier plotting with matplotlib
pdf = grouped_df.to_pandas()

# Calculate number of subplots needed
records_per_plot = 30
total_records = len(pdf)
num_plots = (total_records + records_per_plot - 1) // records_per_plot  # Ceiling division

# Create subplots
fig, axes = plt.subplots(3, 2, figsize=(20, 25))  # 3x2 grid for 6 subplots
axes = axes.flatten()  # Flatten the 2D array of axes for easier indexing

# Color scheme
series_color = '#1f77b4'  # Blue
images_color = '#ff7f0e'  # Orange

# Format numbers with comma as thousand separator
def format_number(x):
    return f'{int(x):,}'

# Plot each subset of data
for plot_idx in range(num_plots):
    ax = axes[plot_idx]
    
    # Get the slice of data for this subplot
    start_idx = plot_idx * records_per_plot
    end_idx = min(start_idx + records_per_plot, total_records)
    plot_data = pdf.iloc[start_idx:end_idx]
    
    # Create horizontal bars
    y_pos = np.arange(len(plot_data))
    
    # Plot series count
    series_bars = ax.barh(y_pos - 0.2, plot_data['total_series'], 
                         height=0.4, color=series_color, label='Series Count')
    
    # Plot image count
    image_bars = ax.barh(y_pos + 0.2, plot_data['total_images'], 
                        height=0.4, color=images_color, label='Image Count')
    
    # Customize the plot
    ax.set_yticks(y_pos)
    ax.set_yticklabels(plot_data['Modality'])
    
    # Add value labels on the bars
    for bars in [series_bars, image_bars]:
        for bar in bars:
            width = bar.get_width()
            ax.text(width, bar.get_y() + bar.get_height()/2,
                   format_number(width),
                   ha='left', va='center', fontsize=8)
    
    # Add shape information as text on the right side
    for idx, (shape, series, images) in enumerate(zip(plot_data['img_array_shape_tuple'], 
                                                     plot_data['total_series'],
                                                     plot_data['total_images'])):
        ax.text(1.02, idx, shape, transform=ax.get_yaxis_transform(),
                ha='left', va='center', fontsize=8)
    
    # Add legend only to the first subplot
    if plot_idx == 0:
        ax.legend()
    
    # Add title
    ax.set_title(f'Subset {plot_idx + 1} (Records {start_idx + 1}-{end_idx})')
    
    # Add grid
    ax.grid(True, axis='x', linestyle='--', alpha=0.3)
    
    # Format x-axis with comma as thousand separator
    ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: format_number(x)))

# Remove any empty subplots
for idx in range(num_plots, len(axes)):
    fig.delaxes(axes[idx])

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
def validate_aneurysm_data(df):
    """
    Validate that for each combination of SeriesInstanceUID, PatientAge, PatientSex, and Modality,
    exactly one of the aneurysm location columns has value 1 and the rest have value 0.
    
    Parameters:
    df (polars.DataFrame): DataFrame with the specified columns
    
    Returns:
    dict: Validation results including violations and summary statistics
    """
    
    # Define the grouping columns (first 4 columns)
    grouping_cols = ['SeriesInstanceUID', 'PatientAge', 'PatientSex', 'Modality']
    
    # Define the aneurysm location columns (remaining 13 columns)
    aneurysm_cols = [
        'Left Infraclinoid Internal Carotid Artery',
        'Right Infraclinoid Internal Carotid Artery',
        'Left Supraclinoid Internal Carotid Artery',
        'Right Supraclinoid Internal Carotid Artery',
        'Left Middle Cerebral Artery',
        'Right Middle Cerebral Artery',
        'Anterior Communicating Artery',
        'Left Anterior Cerebral Artery',
        'Right Anterior Cerebral Artery',
        'Left Posterior Communicating Artery',
        'Right Posterior Communicating Artery',
        'Basilar Tip',
        'Other Posterior Circulation'
    ]
    
    # Check if 'Aneurysm Present' should be included in the validation
    #if 'Aneurysm Present' in df.columns:
    #    aneurysm_cols.append('Aneurysm Present')
    
    print(f"Validating {len(aneurysm_cols)} aneurysm location columns...")
    print(f"Aneurysm columns: {aneurysm_cols}")
    
    # Group by the first 4 columns and sum the aneurysm location columns
    grouped = df.group_by(grouping_cols).agg([
        pl.col(col).sum().alias(f"{col}_sum") for col in aneurysm_cols
    ])
    
    # Calculate the total sum for each group (should be 1 for valid records)
    sum_cols = [f"{col}_sum" for col in aneurysm_cols]
    grouped = grouped.with_columns([
        pl.sum_horizontal(sum_cols).alias('total_aneurysm_count')
    ])
    
    # Find violations (where total is not equal to 1)
    violations = grouped.filter(pl.col('total_aneurysm_count') != 1)
    
    # Separate different types of violations
    zero_violations = grouped.filter(pl.col('total_aneurysm_count') == 0)  # No aneurysm marked
    multiple_violations = grouped.filter(pl.col('total_aneurysm_count') > 1)  # Multiple aneurysms marked
    
    # Create summary statistics
    total_groups = grouped.height
    valid_groups = grouped.filter(pl.col('total_aneurysm_count') == 1).height
    invalid_groups = violations.height
    
    results = {
        'total_groups': total_groups,
        'valid_groups': valid_groups,
        'invalid_groups': invalid_groups,
        'validation_passed': invalid_groups == 0,
        'zero_violations': zero_violations.height,
        'multiple_violations': multiple_violations.height,
        'violations_data': violations,
        'zero_violations_data': zero_violations,
        'multiple_violations_data': multiple_violations
    }
    
    return results

def print_validation_summary(results):
    """Print a summary of the validation results."""
    print("\n" + "="*60)
    print("ANEURYSM DATA VALIDATION SUMMARY")
    print("="*60)
    
    print(f"Total unique combinations: {results['total_groups']}")
    print(f"Valid combinations: {results['valid_groups']}")
    print(f"Invalid combinations: {results['invalid_groups']}")
    print(f"Validation passed: {results['validation_passed']}")
    
    if results['invalid_groups'] > 0:
        print(f"\nTypes of violations:")
        print(f"- No aneurysm marked (sum = 0): {results['zero_violations']}")
        print(f"- Multiple aneurysms marked (sum > 1): {results['multiple_violations']}")
        
        print(f"\nValidation accuracy: {results['valid_groups']/results['total_groups']*100:.2f}%")

def show_violation_details(results, max_violations=10):
    """Show detailed information about violations."""
    
    if results['zero_violations'] > 0:
        print(f"\n{'='*40}")
        print("RECORDS WITH NO ANEURYSM MARKED (sum = 0)")
        print(f"{'='*40}")
        print(results['zero_violations_data'].head(max_violations))
    
    if results['multiple_violations'] > 0:
        print(f"\n{'='*40}")
        print("RECORDS WITH MULTIPLE ANEURYSMS MARKED (sum > 1)")
        print(f"{'='*40}")
        print(results['multiple_violations_data'].head(max_violations))

In [ ]:
results = validate_aneurysm_data(df_aneurysm_present)

In [ ]:
print_validation_summary(results)

In [ ]:
if results['invalid_groups'] > 0:
        show_violation_details(results)

In [ ]:
dd.sql("select * from df_aneurysm_present \
where SeriesInstanceUID = '1.2.826.0.1.3680043.8.498.74614921932700985358270443944241418147' ").pl()